In [ ]:
!pip install accelerate -U
!pip install datasets
!pip install evaluate

In [ ]:
import random
import numpy as np
import pandas as pd
from typing import Dict
from datasets import load_dataset, concatenate_datasets, load_metric, load_dataset
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EvalPrediction,
    Trainer,
    TrainingArguments,
    set_seed,
    EarlyStoppingCallback,
    DataCollatorWithPadding,
)

from sklearn.metrics import accuracy_score, f1_score
import evaluate
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
dataset = load_dataset("tommasobonomo/sem_augmented_fever_nli")
adversarial_testset = load_dataset("iperbole/adversarial_fever_nli")
label_to_id = {'ENTAILMENT': 0, 'NEUTRAL': 1, 'CONTRADICTION': 2}
train_dataset = dataset['train']

In [ ]:
language_model_name = "microsoft/deberta-v3-base"
custom_gsa = 32
batch_size = 8
learning_rate = 4e-5 #4
weight_decay = 0.001
max_inpunt_token_len = 512
epochs = 3

In [ ]:
accuracy_metric = evaluate.load("accuracy", trust_remote_code=True)
f1_metric = evaluate.load("f1", trust_remote_code=True)
precision_metric = evaluate.load("precision", trust_remote_code=True)
recall_metric = evaluate.load("recall", trust_remote_code=True)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')["f1"]
    precision = precision_metric.compute(predictions=predictions, references=labels, average='weighted')["precision"]
    recall = recall_metric.compute(predictions=predictions, references=labels, average='weighted')["recall"]

    return {"accuracy": accuracy, "f1": f1, "precision": precision, "recall": recall}


pos_tag_to_id = {
    'NOUN':3, 'PROPN':4, 'SCONJ':5, 'X':6, 'CCONJ':7, 'PUNCT':8, 'AUX':9, 'NUM':10, 'ADP':11, 'DET':12, 'VERB':13, 'INTJ':14, 'ADJ':15, 'SYM':16, 'ADV':17, 'PRON':18, 'PART':19, "<unk>": 20
    }

def tokenize_function(examples):
    tokens = tokenizer(
        examples["premise"],
        examples["hypothesis"],
        truncation=True,
        max_length=max_inpunt_token_len,
        padding='max_length',
        )

    all_pos_tags_premise = []
    all_pos_tags_hypothesis = []

    for example in examples["wsd"]:
        pos_tags_premise = [pos_tag_to_id.get(token_info["pos"], pos_tag_to_id["<unk>"]) for token_info in example["premise"]]
        pos_tags_hypothesis = [pos_tag_to_id.get(token_info["pos"], pos_tag_to_id["<unk>"]) for token_info in example["hypothesis"]]
        all_pos_tags_premise.append(pos_tags_premise)
        all_pos_tags_hypothesis.append(pos_tags_hypothesis)

    #create a singular pos_tag to insert in the token following the schema of the input_ids:
    #pos_tag = [1, (... pos of premise ... ), 2, (... pos of hypothesis ... ), 2, 0, 0, 0, ..., 0]
    #end zeros are isnerted as padding until len == max_inpunt_token_len
    input_ids_batch = tokens["input_ids"]
    pos_tags = []
    for sample_id in range(len(all_pos_tags_premise)):
        j = 0
        k = 0
        pos_tag_sample = []
        new = False
        for i in input_ids_batch[sample_id]:
            if i == 1:
                pos_tag_sample.append(1)
            elif i == 2:
                pos_tag_sample.append(2)
                if not new:
                    new = True
                else:
                    while len(pos_tag_sample) != max_inpunt_token_len:
                        pos_tag_sample.append(0)
                    pos_tags.append(pos_tag_sample)
                    break
            else:
                if new:
                    if len(all_pos_tags_hypothesis[sample_id]) > k:
                        pos_tag_sample.append(all_pos_tags_hypothesis[sample_id][k])
                        k += 1
                    else:
                        pos_tag_sample.append(0)
                else:
                    if len(all_pos_tags_premise[sample_id]) > j:
                        pos_tag_sample.append(all_pos_tags_premise[sample_id][j])
                        j += 1
                    else:
                        pos_tag_sample.append(0)

    tokens["pos_tags"] = pos_tags

    tokens["label"] = [label_to_id[label] for label in examples["label"]]
    return tokens

In [ ]:
# used for tokenize adv_testset that is without 'wsd' infos
def adv_tokenize_function(examples):
    tokens = tokenizer(
        examples["premise"],
        examples["hypothesis"],
        truncation=True,
        max_length=max_inpunt_token_len,
        padding='max_length',
        )
    l = []
    for example in examples["premise"]:
        l.append([0] * 512)
    tokens["pos_tags"] = l

    tokens["label"] = [label_to_id[label] for label in examples["label"]]
    return tokens

In [ ]:
# pos_tags = set()
# for example in dataset['train']:
#     for ann in example['wsd']['hypothesis']:
#         if ann['pos'] not in pos_tags:
#             pos_tags.add(ann['pos'])
#     for ann in example['wsd']['premise']:
#         if ann['pos'] not in pos_tags:
#             pos_tags.add(ann['pos'])

# for example in dataset['test']:
#     for ann in example['wsd']['hypothesis']:
#         if ann['pos'] not in pos_tags:
#             pos_tags.add(ann['pos'])
#     for ann in example['wsd']['premise']:
#         if ann['pos'] not in pos_tags:
#             pos_tags.add(ann['pos'])

# for example in dataset['validation']:
#     for ann in example['wsd']['hypothesis']:
#         if ann['pos'] not in pos_tags:
#             pos_tags.add(ann['pos'])
#     for ann in example['wsd']['premise']:
#         if ann['pos'] not in pos_tags:
#             pos_tags.add(ann['pos'])

# num_pos_tag = len(pos_tags)
num_pos_tag = 20
# 20 obtained running the code above commented to speedup = 17 + 0, 1, 2 values used for padding e sentence separation

In [ ]:
class CustomModelForSequenceClassification(nn.Module):
    def __init__(self, base_model_name, num_labels):
        super(CustomModelForSequenceClassification, self).__init__()
        self.base_model = AutoModelForSequenceClassification.from_pretrained(
            base_model_name,
            num_labels=num_labels,
            ignore_mismatched_sizes=True,
            output_attentions=False,
            output_hidden_states=False
        )
        self.pos_tag_embeddings = nn.Embedding(num_pos_tag, self.base_model.config.hidden_size)
        self.classifier = nn.Linear(self.base_model.config.hidden_size, num_labels)
        self.hidden_size = self.base_model.config.hidden_size
        self.linear_layer = nn.Linear(self.hidden_size * 2, self.hidden_size).to(device)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, pos_tags=None, labels=None):

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        pos_tags = pos_tags.to(device)
        labels = labels.to(device)

        outputs = self.base_model.base_model(input_ids=input_ids,
                                             attention_mask=attention_mask,
                                             token_type_ids=token_type_ids)

        hidden_states = outputs[0]

        pos_tag_embeds = self.pos_tag_embeddings(pos_tags)

        combined_embeds = self.linear_layer(torch.cat((hidden_states, pos_tag_embeds), dim=-1))

        pooled_output = torch.mean(combined_embeds, dim=1)

        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)

        return (loss, logits) if loss is not None else logits

model = CustomModelForSequenceClassification(
    base_model_name=language_model_name,
    num_labels=3,
)

In [ ]:
set_seed(42)
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

tokenizer = AutoTokenizer.from_pretrained(language_model_name)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=dataset["train"].column_names, num_proc=2)
tokenized_eval_dataset = dataset["validation"].map(tokenize_function, batched=True, remove_columns=dataset["train"].column_names, num_proc=2)

torch.cuda.empty_cache()

In [ ]:
training_args = TrainingArguments(
    output_dir="training_dir",
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=custom_gsa,# Accumulates gradients over custom_gsa steps
    warmup_steps=500,                      # number of warmup steps for learning rate scheduler
    weight_decay=weight_decay,
    save_strategy="epoch",
    learning_rate=learning_rate,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    logging_steps=100,
    fp16=True,                             # Enable mixed precision training to save memory
    report_to="none",
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",     # load best model based on best evaluation loss
    dataloader_num_workers=2,
)

In [ ]:
set_seed(42)
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)
torch.cuda.empty_cache()
trainer.train()
torch.cuda.empty_cache()

In [ ]:
train_result = trainer.evaluate()

In [ ]:
trainer.save_model()
trainer.log_metrics("train", train_result)
trainer.save_state()
logs = trainer.state.log_history

In [ ]:
test_dataset = dataset["test"]
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=dataset["test"].column_names, num_proc=2, cache_file_name='test_cache.arrow')
test_result = trainer.evaluate(eval_dataset=tokenized_test_dataset)
trainer.log_metrics("test", test_result)
print(test_result)

In [ ]:
tokenized_adversarial_testset = adversarial_testset.map(adv_tokenize_function, batched=True, remove_columns=['part'], num_proc=2)
adversarial_test_result = trainer.evaluate(eval_dataset=tokenized_adversarial_testset)
trainer.log_metrics("adversarial_test", adversarial_test_result)
print(adversarial_test_result)